<div style="text-align: center; font-size: 40px; >
    <strong>Industry Data Analysis</strong>
</div>


<div style="text-align: center; font-size: 40px;">
    <strong>Industry Data Analysis</strong>
</div>


## **Introduction**
This report analyzes the registered company’s dataset, which contains details of all companies registered in India from 1857 to 2020. The dataset includes information on company status, business activity, registration date, capital, and location. The analysis focuses on understanding trends in business activity, company growth, and regional variations, providing insights into the Indian corporate landscape over time.
>


### **Importing the Libraries**


In [4]:
import pandas as pd
from sqlalchemy import create_engine
from urllib.parse import quote_plus

## **DB Schema Diagram**
- The ER diagram represents the DB schema followed for the analysis showing relations between various entities.

<div style="text-align: center;">
    <img src="https://mermaid.ink/img/pako:eNqVVE1vozAQ_StozkkUCB-F82ovvay0t4rL1J4Sa8FGtlmVTfPf1_mgIeC0CSeYZ7-ZefOYHTDFCQog_UNgpbEpZeAeppoWZc_JoqhNsDtFD4-xWsgqYEIGv57n4dM9iQ1dMCGtw9F2RvDg5_M1wGo03jhaqpTuPZDpXm-g3IUDTZVw5aAVSo4gYqLBOsDObhm2wmI9q15IfixnBjhK5FyTB6LGKTSpwHFYTbbT0gMclKCGpJ0LRJN2zilenXCSvCoNveqpEOduWxT8qtt9Ka8mPAg5HvFEfs-YL_Od8R30m5GdZ_wQ02drU7Zxzw8xnjw4pft0ptfOnGZBTobdSHAx5izLlWfvK7t1ABMt1oMBkFnxV9je8z-OPHIf-UlF0sSPzvOpQg8Pf2j-42O5VLvpFimCLRqPW-49fXHErRsNSqzI-KZ-b5LxDL-9c3tC316d6v_FBVhAQ9rtGe4W9XFOJdit2yElFO6Vo_5TQin37pzbbup3LxkUVne0AK26agvFG9bGfXXtYQudF_1wxKV7UWr8CcUO3qFYhtF6tcnTdZxleZKncZgsoIci3GSrJM2f1lGWRFEehel-Af-OFNEq3cR5koZRGMdxto72_wFX1Q1I" width="900"/></div>

- The database schema organizes company information with CompanyDetails as the central table.
- CompanyDetails contains attributes like CIN (Primary Key), name, registration date, capital, address, and references to related tables.
- Lookup tables and their connections:
  - **CompanyCategory**: Linked via categoryid to define the company category.
  - **CompanySubCategory**: Connected through subcategoryid for subcategories of the company.
  - **CompanyClass**: Linked via classid to specify the company class.
  - **CompanyStatus**: Connected through statusid to indicate the operational status.
  - **PrincipalBusinessActivity**: Linked via businessid to identify the company's main activity.
  - **RegisteredState**: Connected via stateid to specify the registration state.
  - **CompanyRegistrar**: Linked through registrarid to associate the managing registrar.
- These relationships use foreign keys in CompanyDetails, ensuring normalized data and enabling efficient cross-referencing and querying.



### **DB Connection Function**

Acts as the bridge for connecting to the database and executing queries.


In [3]:
def get_alchemy_engine():
    """
    Creates and returns an SQLAlchemy engine using the database settings.
    """
    db_settings = {
        "ENGINE": "django.db.backends.postgresql",  # Database engine
        
        "NAME": "IndustriesDB",                     # Database name
        "USER": "postgres",                        # Username
        "PASSWORD": "password@123",                # Password with special characters
        "HOST": "localhost",                       # Host address
        "PORT": "5432",                            # Port
    }
    
    # URL-encode the password to handle special characters
    encoded_password = quote_plus(db_settings['PASSWORD'])
    
    # Construct the SQLAlchemy connection URL
    connection_url = (
        f"postgresql://{db_settings['USER']}:{encoded_password}@"
        f"{db_settings['HOST']}:{db_settings['PORT']}/{db_settings['NAME']}"
    )
    
    # Create and return the SQLAlchemy engine
    engine = create_engine(connection_url)
    return engine

def fetch_data(query):
    """
    Fetches data from the PostgreSQL database using Pandas.
    :param query: SQL query string
    :return: Pandas DataFrame with query results
    """
    engine = get_alchemy_engine()
    
    # Use Pandas to execute the query and fetch data into a DataFrame
    with engine.connect() as connection:
        df = pd.read_sql(query, connection)

    if engine:
        engine.dispose()
        # print("Engine connection closed.")
        
    return df

## **Data Feature Description**
For the dataset considered in the analysis following is the detailed overview for the same, representing important info for each feature.
                                                                                                                       

In [9]:
descQuery = '''
    SELECT
        UPPER(column_name) AS "Feature Name",
        UPPER(data_type) AS "Feature Type",
        CASE 
            WHEN data_type IN ('character varying', 'text') THEN 'VARCHAR'
            WHEN data_type IN ('integer', 'bigint', 'smallint') THEN 'INTEGER'
            WHEN data_type = 'date' THEN 'DATE'
            ELSE data_type
        END AS "Feature DTYPE",
        (SELECT COUNT(*) 
         FROM companydetails 
         WHERE column_name IS NULL) AS "Total Null Values"
    FROM
        information_schema.columns
    WHERE
        table_name = 'companydetails';
'''
data = fetch_data(descQuery)
data  

,Feature Name,Feature Type,Feature DTYPE,Total Null Values
0,PAIDUPCAPTIAL,BIGINT,INTEGER,0
1,LASTFINANCIALSTATEMENT,DATE,DATE,0
2,STATEID,BIGINT,INTEGER,0
3,BUSINESSID,BIGINT,INTEGER,0
4,REGISTRARID,BIGINT,INTEGER,0
5,STATUSID,BIGINT,INTEGER,0
6,CLASSID,BIGINT,INTEGER,0
7,CATEGORYID,BIGINT,INTEGER,0
8,SUBCATEGORYID,BIGINT,INTEGER,0
9,DATEOFREGISTRATION,DATE,DATE,0


- **Total Size of the dataset**: 19,92,170
- The columns **INDUSTRIALCLASS, LASTANNUALRETURN, and LASTFINANCIALSTATEMENT Date** are excluded from analysis as they lack descriptive context or do not contribute significant insights to the current objectives.

## **Exploratory Data Analysis**

### **Basic Statistics and Counts:**
   - The following queries provide an overview of the data, summarizing key counts and distributions of companies.

In [17]:
nationalCountQuery = '''
    SELECT 
        COUNT(*) as totalCount 
    FROM companydetails cd
'''

stateCountQuery = '''
    SELECT 
    	rs.statename ,
     	COUNT(cd.cin)
	FROM 
    	companydetails cd
	JOIN registeredstate rs 
	ON rs.stateid  = cd.stateid 
	GROUP BY 
    	rs.statename 
    ORDER BY 
        COUNT(cd.cin)
    DESC

'''

In [18]:
nationalCount = fetch_data(nationalCountQuery)
stateCount = fetch_data(stateCountQuery)  

nationalCount

,totalcount
0,1989645


- Total number of companies registered: 19,92,170

In [25]:
stateCount.head()

,statename,count
0,Maharashtra,394932
1,Delhi,346575
2,West Bengal,206985
3,Tamil Nadu,150832
4,Karnataka,125692


In [26]:
stateCount.tail()

,statename,count
31,Andaman and Nicobar Islands,480
32,Daman and Diu,397
33,Mizoram,175
34,Lakshadweep,18
35,Sikkim,2


- State wise company registered:
  - Maharashtra has the most companies registered in the given timeline with 3,94,932 which is 19.20% of the total.
  -	Maharashtra, Delhi, West Bengal, Tamil Nadu, Karnataka, Uttar Pradesh, Telangana, Gujarat add to the 78% of all the companies registered.
  -	Sikkim is the least registered state with 2 companies.
  -	Among the UTs after Delhi, Chandigarh has the most companies registered 15,018.
  -	Lakshadweep has the least registration totalling to 18.


### **Total Companies by Status:**

In [20]:
nationalStatusQuery = '''
    SELECT 
        cs.statuscode ,
        cs.statusdesc ,
        COUNT(cin) as companyCount
    from companydetails cd 
    JOIN 
        companystatus cs 
    ON cd.statusid = cs.statusid 
    WHERE dateofregistration IS NOT NULL 
    group by 
        cs.statuscode ,
        cs.statusdesc  
    order by 
        COUNT(cin) desc
'''

stateStatusQuery = '''
SELECT 
    t1.*,
    ROUND((CAST(t1.activeCount AS DECIMAL(10, 2)) / t1.totalCompanies)*100) as activePercentage,
    ROUND((CAST(t1.dormantCount AS DECIMAL(10, 2)) / t1.totalCompanies)*100) as dormantPercentage
FROM 
(
	SELECT 
    	rs.statename ,
    	COUNT(cd.cin) AS totalCompanies,
    	SUM(
    	CASE 
    		WHEN cs.statuscode IN  ('ACTV','AMAL','CLLP') THEN 1 ELSE 0
    	END
    	) AS activeCount,
    	SUM(
    	CASE 
    		WHEN cs.statuscode NOT IN  ('ACTV','AMAL','CLLP') THEN 1 ELSE 0
    	END
    	) AS dormantCount
	FROM companydetails cd 
	JOIN 
    	companystatus cs 
	ON cd.statusid = cs.statusid 
	JOIN 
         registeredstate rs
	ON cd.stateid = rs.stateid 
	WHERE dateofregistration IS NOT NULL 
	GROUP BY 
    	rs.statename
	ORDER BY 
    	COUNT(cin) DESC
) AS t1
'''

In [21]:
nationalStatusCount = fetch_data(nationalStatusQuery)
stateStatusCount = fetch_data(stateStatusQuery)  

nationalStatusCount

,statuscode,statusdesc,companycount
0,ACTV,Active,1189527
1,STOF,Strike off,687419
2,UPSO,Under process of Striking Off,41456
3,AMAL,Amalgamated,24887
4,CLLP,Converted to LLP,13175
5,DISD,Dissolved,9766
6,NAEF,Not available for e-filing,8815
7,ULQD,Under liquidation,6458
8,CLLD,Converted to LLP and Dissolved,4874
9,D455,Liquidated,2145


- **Total Companies by Status:**
  - Registered companies’ status wise distribution: 
    - As of 2020 of the total companies registered 60% of them are active in their operations.
    - From the remaining around 38% are non operational with most falling in the Strike-Off status, while others are categorized as Dissolved, Under Liquidation, Dormant, or in the process of Striking Off, indicating various stages of closure or inactivity.

In [23]:
stateStatusCount.head()

,statename,totalcompanies,activecount,dormantcount,activepercentage,dormantpercentage
0,Maharashtra,394932,242477,152455,61.0,39.0
1,Delhi,346575,211165,135410,61.0,39.0
2,West Bengal,206985,135959,71026,66.0,34.0
3,Tamil Nadu,150832,80594,70238,53.0,47.0
4,Karnataka,125692,79634,46058,63.0,37.0


In [24]:
stateStatusCount.tail()

,statename,totalcompanies,activecount,dormantcount,activepercentage,dormantpercentage
31,Andaman and Nicobar Islands,480,354,126,74.0,26.0
32,Daman and Diu,397,268,129,68.0,32.0
33,Mizoram,175,101,74,58.0,42.0
34,Lakshadweep,18,11,7,61.0,39.0
35,Sikkim,2,2,0,100.0,0.0


- **Total Companies by Status:**
  - Registered companies’ status and State wise distribution:
    - Most operational companies are in industrialized and economically stronger states such as Delhi, Maharashtra, Karnataka, Gujarat, and Tamil Nadu.
    - States like Punjab, Pondicherry, Nagaland, Arunachal Pradesh, and Chandigarh reflect higher dormant percentages, indicating possible stagnation or regulatory hurdles.
    - Regions like Sikkim, Dadra and Nagar Haveli, and Andaman & Nicobar Islands show higher percentages of operational companies, likely due to smaller total numbers and better governance.

### **Total Companies by Class:**

In [32]:
nationalClassQuery = '''
    SELECT 
        cc.classname,
    	COUNT(cin)
    FROM
    	companydetails cd 
    JOIN 
    	companyclass cc
    ON cd.classid = cc.classid 
    GROUP BY 
    	cc.classname 
    ORDER BY	 
    	count(cin) desc
'''

In [36]:
nationalClassCount = fetch_data(nationalClassQuery)
nationalClassCount

,classname,count
0,Private,1817740
1,Public,137460
2,Private(One Person Company),30216
3,,4229


- Private Companies dominate with 1,817,740 entries, reflecting the majority of the entries and suggesting high private sector dominance and potential for small business growth analysis.
- Public Companies have 137,460 entries, significantly fewer than private ones.
- Private (One Person Company) accounts for 30,216 entries, indicating entrepreneurial activity.

In [37]:
stateClassQuery = '''
    SELECT 
    	rs.statename ,
    	COUNT(cin),
    	SUM(
    	    CASE 
    	    	WHEN cc.classname = 'Private' THEN 1 ELSE 0
    	    END
        ) AS privateCount,
        SUM(
    	    CASE 
    	    	WHEN cc.classname = 'Public' THEN 1 ELSE 0
    	    END
        ) AS publicCount,
        SUM(
    	    CASE 
    	    	WHEN cc.classname = 'Private(One Person Company)' THEN 1 ELSE 0
    	    END
        ) AS singlePrivateCount
    
    FROM
    	companydetails cd 
    JOIN 
    	companyclass cc
    ON cd.classid = cc.classid 
    JOIN 
    	registeredstate rs
    ON cd.stateid = rs.stateid 
    GROUP BY 
    	rs.statename 
    ORDER BY	 
        count(cin) DESC
'''

In [39]:
stateClassCount = fetch_data(stateClassQuery)
stateClassCount.head()

,statename,count,privatecount,publiccount,singleprivatecount
0,Maharashtra,394932,359256,29376,5252
1,Delhi,346575,320891,20188,3894
2,West Bengal,206985,186769,18987,1151
3,Tamil Nadu,150832,137173,11237,2127
4,Karnataka,125692,117309,4811,3191


In [40]:
stateClassCount.tail()

,statename,count,privatecount,publiccount,singleprivatecount
31,Andaman and Nicobar Islands,480,460,9,11
32,Daman and Diu,397,318,73,6
33,Mizoram,175,149,17,9
34,Lakshadweep,18,17,1,0
35,Sikkim,2,2,0,0


#### **Top States:**
- Maharashtra leads with 394,932 companies (most private: 359,256).
- Delhi is second with 346,575 companies, showcasing a strong business presence.
- West Bengal ranks third with 206,985 companies, emphasizing its economic activity.

#### **Private Companies:**
- Predominantly concentrated in Maharashtra, Delhi, and West Bengal.
- Reflects strong entrepreneurial activity in these regions.

#### **Public Companies:**
- Highest in Maharashtra (29,376), followed by Delhi (20,188) and West Bengal (18,987).

#### **Single-Person Private Companies:**
- Most common in Maharashtra (5,252) and Delhi (3,894), indicating individual-driven enterprises.

#### **Smaller States:**
- States like Lakshadweep, Sikkim, and Mizoram have minimal business presence.
- Sikkim has only 2 companies, and Lakshadweep has just 18 companies.

#### **Balanced Growth:**
- Southern states like Tamil Nadu, Karnataka, and Kerala show moderate company distribution, reflecting diverse business opportunities.

#### **North-Eastern States:**
- Minimal corporate presence compared to other regions, highlighting potential for economic development.

#### **Unique Trends:**
- Union Territories like Chandigarh (15,018 companies) and Dadra and Nagar Haveli (550 companies) show varying business densities.

### **Total Companies by Category and SubCategory:**

In [48]:
nationalCategoryQuery = '''
    SELECT 
    cc.categoryname ,
    COUNT(cin) as totalCount,
    SUM(
    	CASE 
    		WHEN csc.categoryname = 'Subsidiary of Foreign Company' THEN 1 ELSE 0
    	END 
       ) AS "Subsidiary of Foreign Company",
    SUM(
    	CASE 
    		WHEN csc.categoryname = 'Guarantee and Association comp' THEN 1 ELSE 0
    	END 
       ) AS "Guarantee and Association comp",
    SUM(
    	CASE 
    		WHEN csc.categoryname = 'State Govt company' THEN 1 ELSE 0
    	END 
       ) AS "State Govt company",
	SUM(
		CASE 
			WHEN csc.categoryname = 'Union Govt company' THEN 1 ELSE 0
		END 
	   ) AS "Union Govt company",
	SUM(
		CASE 
			WHEN csc.categoryname = 'Non-govt company' THEN 1 ELSE 0
		END 
	   ) AS "Non-govt company"
    FROM 
        companydetails cd 
    JOIN 
        companycategory cc
    ON cd.categoryid = cc.categoryid 
    JOIN
        companysubcategory csc 
    ON cd.subcategoryid = csc.subcategoryid 
    WHERE cc.categoryname != ''
    GROUP BY 
        cc.categoryname 
    --    csc.categoryname 
    ORDER BY 
        cc.categoryname ,
    --    csc.categoryname ,
        count(cin)
    DESC

'''

In [49]:
nationalCatCount = fetch_data(nationalCategoryQuery)
nationalCatCount

,categoryname,totalcount,Subsidiary of Foreign Company,Guarantee and Association comp,State Govt company,Union Govt company,Non-govt company
0,Company Limited by Guarantee,20563,9,2308,18,4,18224
1,Company limited by Shares,1963881,12344,69,1889,571,1949004
2,Unlimited Company,967,16,7,5,1,938



#### **Limited by Guarantee:**
- Total: 20,563
- Dominated by non-government companies (88.63%).
- Very few foreign subsidiaries (0.04%) and government companies (0.11%).

#### **Limited by Shares:**
- Total: 1,963,881
- Majority are non-government (99.24%).
- Small presence of foreign subsidiaries (0.63%) and government companies (0.13%).

#### **Unlimited Companies:**
- Total: 967
- Primarily non-government (97%).
- Highest proportion of foreign subsidiaries (1.65%) among categories.

#### **Ownership Highlights:**
- Non-Government Companies: Predominant across all types (88.63% to 99.24%).
- Foreign Subsidiaries: Highest in Unlimited Companies (1.65%).
- Government Companies: Minimal representation across all categories (<0.2%).

#### **Key Insight:**
- Non-Government Companies overwhelmingly dominate, with minimal foreign and government ownership.

In [50]:
stateCategoryQuery = '''
    SELECT 
	rs.statename ,
    cc.categoryname ,
    COUNT(cin),
    SUM(
    	CASE 
    		WHEN csc.categoryname = 'Subsidiary of Foreign Company' THEN 1 ELSE 0
    	END 
       ) AS "Subsidiary of Foreign Company",
    SUM(
    	CASE 
    		WHEN csc.categoryname = 'Guarantee and Association comp' THEN 1 ELSE 0
    	END 
       ) AS "Guarantee and Association comp",
    SUM(
    	CASE 
    		WHEN csc.categoryname = 'State Govt company' THEN 1 ELSE 0
    	END 
       ) AS "State Govt company",
	SUM(
		CASE 
			WHEN csc.categoryname = 'Union Govt company' THEN 1 ELSE 0
		END 
	   ) AS "Union Govt company",
	SUM(
		CASE 
			WHEN csc.categoryname = 'Non-govt company' THEN 1 ELSE 0
		END 
	   ) AS "Non-govt company"
    FROM 
        companydetails cd 
    JOIN 
        companycategory cc
    ON cd.categoryid = cc.categoryid 
    JOIN
        companysubcategory csc 
    ON cd.subcategoryid = csc.subcategoryid 
    JOIN 
    	registeredstate rs
    ON cd.stateid = rs.stateid 
    WHERE cc.categoryname != ''
    GROUP BY 
    	rs.stateid ,
        cc.categoryname 
    --    csc.categoryname 
    ORDER BY 
    	rs.statename ,
        cc.categoryname ,
    --    csc.categoryname ,
        count(cin)
    DESC

'''

In [52]:
stateCatCount = fetch_data(stateCategoryQuery)
stateCatCount.head()

,statename,categoryname,count,Subsidiary of Foreign Company,Guarantee and Association comp,State Govt company,Union Govt company,Non-govt company
0,Andaman and Nicobar Islands,Company Limited by Guarantee,1,0,0,0,0,1
1,Andaman and Nicobar Islands,Company limited by Shares,479,0,0,2,1,476
2,Andhra Pradesh,Company Limited by Guarantee,484,0,8,1,0,475
3,Andhra Pradesh,Company limited by Shares,32432,145,0,75,4,32205
4,Andhra Pradesh,Unlimited Company,11,0,0,0,0,11


In [53]:
stateCatCount.tail()

,statename,categoryname,count,Subsidiary of Foreign Company,Guarantee and Association comp,State Govt company,Union Govt company,Non-govt company
93,Uttaranchal,Company limited by Shares,8883,10,0,25,4,8844
94,Uttaranchal,Unlimited Company,5,0,0,0,0,5
95,West Bengal,Company Limited by Guarantee,701,0,237,0,0,464
96,West Bengal,Company limited by Shares,206175,151,2,89,47,205886
97,West Bengal,Unlimited Company,31,0,0,0,0,31


#### **State-wise Ownership Highlights:**
- Foreign Subsidiaries: More prominent in states like Delhi, Maharashtra, and Gujarat.
- Government Companies (State & Union): Generally low across all states, with slight peaks in Delhi and Maharashtra.
- Non-Government Companies: Dominant across all company types and states.

#### **Notable Data Points:**
- Delhi has a higher concentration of foreign subsidiaries (2384 in Company Limited by Shares).
- Maharashtra and Gujarat have a significant number of non-government companies.
- Andaman and Nicobar Islands, Arunachal Pradesh, and Lakshadweep show very few companies, especially in the "Company Limited by Guarantee" category.

#### **Key Trends:**
- Non-Government Companies are overwhelmingly dominant across all states.
- Foreign subsidiaries are most common in large states like Delhi and Maharashtra.
- Government representation remains minimal, with a higher concentration in Delhi.

### **Total Companies by Business Type**

#### **The following business activities have been analyzed at the national and state levels, with short names for easy reference:**

- Real Estate Renting and Business Activities – Real Estate
- Manufacturing – Manufacturing
- Wholesale and Retail Trade; Repair of Motor Vehicles, Motorcycles, and Personal and Household Goods – Retail & Wholesale
- Construction – Construction
- Financial Intermediation – Finance
- Agriculture & Allied – Agriculture
- Transport, Storage, and Communications – Transport & Communication
- Other Community, Social, and Personal Service Activities – Social Services
- Hotels and Restaurants – Hospitality
- Extraterritorial Organizations and Bodies – Extraterritorial
- Health and Social Work – Health & Social Work
- Education – Education
- Electricity, Gas, and Water Supply – Utilities
- Mining and Quarrying – Mining
- Public Administration and Defence; Compulsory Social Security – Public Admin
- Unclassified – Unclassified
- Activities of Private Households as Employers and Undifferentiated Production Activities of Private Households – Household Activities

##### **These sectors are analyzed to identify regional trends and economic patterns across states and the nation.**

In [6]:
businessQuery = '''
    SELECT
    	*,
    	(SELECT businesscode FROM tempTable WHERE statename = t1.statename AND companycount = t1.maxcount LIMIT 1) AS maxbusiness,
    	(SELECT businesscode FROM tempTable WHERE statename = t1.statename AND companycount = t1.mincount LIMIT 1) AS minbusiness
    FROM 
    (
    	SELECT 
    		t1.statename,
    		MAX(companyCount) AS maxcount,
    		MIN(companyCount) AS mincount,
    		COUNT(*) AS totalBusinessTypes
    	FROM 
    		tempTable AS t1
    	GROUP BY 
    		t1.statename
    ) AS t1

'''

In [7]:
businessData = fetch_data(businessQuery)
businessData.head()

,statename,maxcount,mincount,totalbusinesstypes,maxbusiness,minbusiness
0,Rajasthan,15359,2,17,Real Estate,Household Activities
1,Jharkhand,3955,1,17,Real Estate,Unclassified
2,Maharashtra,128201,3,18,Real Estate,None
3,Lakshadweep,5,1,8,Agriculture,Extraterritorial
4,Bihar,9730,7,16,Real Estate,Household Activities


#### **Dominant Business Activity:**
  - Real Estate leads across most states.

#### **Highest and Lowest Business Counts:**
  - Delhi has the most businesses (152,745) in Real Estate, while Lakshadweep has the least (5) in Agriculture.

#### **Business Diversity:**
  - States like Maharashtra, Haryana, and Delhi have the most diverse businesses (18 types). Lakshadweep has the least diversity (8 types).

#### **Balanced Distribution:**
  - Karnataka and West Bengal have high minimum counts (24) across categories, indicating balanced business activity.

#### **Emerging and Specialized States:**
  - Sikkim and Lakshadweep focus on niche categories like Transport & Communication and Agriculture.

#### **Uncommon Categories:**
  - Unclassified and Extraterritorial businesses are among the least common types.

#### **Regional Observations:**
  - Northeastern states have fewer businesses, with Manufacturing and Real Estate dominating.

#### **Significant Differences:**
  - States like Maharashtra show wide gaps between max (128,201) and min (3) counts in business types.

In [8]:
businessPerformanceQuery = '''
    SELECT
	*,
    	(SELECT businesscode FROM tempTableV2 WHERE statename = t1.statename AND companycount = t1.maxcount LIMIT 1) AS mostSuccessbusiness,
    	(SELECT businesscode FROM tempTableV2 WHERE statename = t1.statename AND companycount = t1.mincount LIMIT 1) AS leastSuccessbusiness
    FROM 
    (
    	SELECT 
    		t1.statename,
    		MAX(companyCount) AS maxcount,
    		MIN(companyCount) AS mincount
    	FROM 
    		tempTableV2 AS t1
    	GROUP BY 
    		t1.statename
    ) AS t1

'''

In [10]:
businessperformanceData = fetch_data(businessPerformanceQuery)
businessperformanceData

,statename,maxcount,mincount,mostsuccessbusiness,leastsuccessbusiness
0,Rajasthan,10148,1,Real Estate,Household Activities
1,Jharkhand,2957,1,Real Estate,Unclassified
2,Maharashtra,86807,2,Real Estate,None
3,Lakshadweep,4,1,Agriculture,Extraterritorial
4,Bihar,7363,2,Real Estate,Extraterritorial
5,Gujarat,25386,4,Manufacturing,None
6,Punjab,5679,3,Manufacturing,Public Admin
7,Himachal Pradesh,899,3,Real Estate,Public Admin
8,Nagaland,62,1,Real Estate,Health & Social Work
9,Karnataka,39004,8,Real Estate,Unclassified


#### **Most Successful Business Activity:**
 - Real Estate dominates as the most successful business across most states.

#### **Least Successful Business Activity:**
 - Unclassified and Extraterritorial are the least successful business types in multiple states.

#### **Highest Success Count:**
 - Delhi leads with 97,760 successful businesses in Real Estate, followed by Maharashtra (86,807).

#### **Lowest Success Count:**
 - Sikkim has only 1 successful business, entirely in Real Estate.

#### **Business Diversity:**
 - States like Dadra and Nagra Haveli, Lakshadweep, and Tripura have low success counts in niche categories like Manufacturing, Agriculture, and Mining.

#### **Specialized States:**
- West Bengal shows success in Retail & Wholesale (38,593 businesses), indicating a unique business trend.

#### **Balanced States:**
- States like Karnataka (min: 8) and Uttar Pradesh (min: 9) have relatively high minimum counts, reflecting a balanced business landscape.

#### **Emerging Regions:**
- Smaller states like Nagaland and Mizoram are emerging with Real Estate as their primary success driver but have limited success in other categories.

#### **Sector Observations:**
- Manufacturing remains prominent in industrial states like Gujarat and Punjab, but Public Administration appears as the least successful business in these areas.